Original https://ts.gluon.ai/stable/tutorials/forecasting/quick_start_tutorial.html

In [1]:
!pip install --upgrade mxnet
!pip install gluonts
!pip install orjson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 16.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 4.4 MB/s eta 0:00:00


In [2]:
import gluonts, orjson
#from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
import numpy as np
import pandas as pd
from gluonts.dataset.common import ListDataset
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

Целевые данные без признаков

In [7]:
# Загружаем данные
df = pd.read_csv('/content/drive/MyDrive/trainee/Ingate/Ozon/data_ozon.csv',
                 parse_dates=['День'])
dic_ren = {'День': 'timestamp', 'Ozon ID': 'item_id', 'Заказано товаров': 'target'}
df = df.rename(columns=dic_ren)
freq = 'D'
pred_len = 24
df = df[['timestamp', 'target', 'item_id']]
df_tmp = df.groupby('item_id').sum()
ls_work = df_tmp[df_tmp > 100].index.tolist()
df_tmp = df[df.item_id.isin(ls_work)]
df = df_tmp[df_tmp.target > 0]
df = df.pivot(index='item_id', columns='timestamp', values='target')
df.fillna(0, inplace=True)
df.reset_index(inplace=True)
df = df.melt(('item_id'), var_name='timestamp', value_name='target')
# Сортируем по датам и SKU
df.sort_values(['timestamp', 'item_id'])
# Индексируем по дате
df.set_index('timestamp')
ds = PandasDataset.from_long_dataframe(df, target='target', item_id='item_id')
train_ds, val_ds = split(ds, offset=pred_len * -2)
trainer = Trainer(epochs=20, learning_rate=1e-3)
estimator = DeepAREstimator(prediction_length=pred_len, freq=freq, trainer=trainer)
model = estimator.train(train_ds)

<ipython-input-7-9471553fa7bb>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/trainee/Ingate/Ozon/data_ozon.csv',
<ipython-input-7-9471553fa7bb>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_tmp = df.groupby('item_id').sum()
100%|██████████| 50/50 [00:06<00:00,  8.27it/s, epoch=20/20, avg_epoch_loss=-6.23]
